# ScfindME: Mining Alternative Splicing Patterns in Large Single-cell Atlases

## Example analysis workflow
**Author:**  
Yuyao Song  
**Date:**  
May 2022  

## 1. Set up

### 1.1 Install package

In [1]:
# to install the latest release from GitHub
# install.packages("devtools")
# devtools::install_github("hemberg-lab/scfindME", force = TRUE)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done




cli          (3.2.0  -> 3.3.0) [CRAN]
vctrs        (0.3.8  -> 0.4.1) [CRAN]
magrittr     (2.0.2  -> 2.0.3) [CRAN]
fansi        (1.0.2  -> 1.0.3) [CRAN]
jsonlite     (1.7.3  -> 1.8.0) [CRAN]
checkmate    (2.0.0  -> 2.1.0) [CRAN]
knitr        (1.37   -> 1.39 ) [CRAN]
tzdb         (NA     -> 0.3.0) [CRAN]
dplyr        (1.0.8  -> 1.0.9) [CRAN]
openssl      (1.4.6  -> 2.0.0) [CRAN]
ps           (1.6.0  -> 1.7.0) [CRAN]
tinytex      (0.31   -> 0.38 ) [CRAN]
processx     (3.5.2  -> 3.5.3) [CRAN]
rmarkdown    (2.7    -> 2.14 ) [CRAN]
clipr        (0.7.1  -> 0.8.0) [CRAN]
vroom        (NA     -> 1.5.7) [CRAN]
broom        (0.7.12 -> 0.8.0) [CRAN]
readr        (1.4.0  -> 2.1.2) [CRAN]
uuid         (0.1-4  -> 1.1-0) [CRAN]
googledrive  (1.0.1  -> 2.0.0) [CRAN]
gargle       (1.1.0  -> 1.2.0) [CRAN]
blob         (1.2.2  -> 1.2.3) [CRAN]
plyr         (1.8.6  -> 1.8.7) [CRAN]
rvest        (1.0.0  -> 1.0.2) [CRAN]
reprex       (2.0.0  -> 2.0.1) [CRAN]
readxl       (1.3.1  -> 1.4.0) [CRAN]
haven       

Skipping 1 packages not available: biomaRt

Installing 30 packages: cli, vctrs, magrittr, fansi, jsonlite, checkmate, knitr, tzdb, dplyr, openssl, ps, tinytex, processx, rmarkdown, clipr, vroom, broom, readr, uuid, googledrive, gargle, blob, plyr, rvest, reprex, readxl, haven, googlesheets4, dtplyr, Hmisc

Warning message in i.p(...):
“installation of package ‘magrittr’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘fansi’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘jsonlite’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘openssl’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘vctrs’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘processx’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘dplyr’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.h

✔  checking for file ‘/tmp/Rtmpke5svQ/remotes140f3960acfc2/hemberg-lab-scfindME-b9b749c/DESCRIPTION’ (416ms)
─  preparing ‘scfindME’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  excluding invalid files
   Subdirectory 'R' contains invalid file names:
     ‘ME_analysis.ipynb’
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘scfindME_0.1.0.tar.gz’
   


In [60]:
getwd()

[1] "/nfs/research/irene/ysong/DATA/SCFIND/scfindME_package/scfindME/vignettes"

In [74]:
devtools::load_all("/nfs/research/irene/ysong/DATA/SCFIND/scfindME_package/scfindME/")

ℹ Loading scfindME



### 1.2 Load packages

In [1]:
# load the package scfindME
# also load packages for data manipulation and visulisation
library(scfindME)
library(tibble)
library(dplyr)
library(tidyr)
library(ggplot2)
library(repr)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# a useful function for setting figure size

library(repr)
fig <- function(width, heigth){
     options(repr.plot.width = width, repr.plot.height = heigth)
}

### 1.3 Load example index

In [3]:
# These are the pre-built indices using the Tasic et al.(2016) dataset
# The dataset have detailed subtype classifications of glutamatergic neurons and GABA-ergic neurons in the mouse primary visual cortex

index <- loadObject("../../processed_data/Tasic_2016/tasic_2016_index.rds")

86 MB 
Version 6


In [4]:
# the index is a SCFind class object
class(index)

# the object contains four attributes
slotNames(index)

# the metadata attribute further stores splicing-related information for comprehending the results
## stats stores the mean and SD of the PSI for each node in the index
## node_list stores node related annotations
## diff_cut is a binary matrix to store if the difference of the PSI in a particular pesudobulk and the mean PSI across all pseudobulks is above the cut-off during index building (default 0.2)
## this is to distinguish between PSI = NA (the junction is unquantified), or the PSI of a node is not sufficiently different from dataset mean (PSI ~ mean_PSI) during raw PSI retrieval from index. Both circumstance is treated as 0 in the splicing matrix and compressed during index building

names(index@metadata)

[1] "SCFind"
attr(,"package")
[1] "scfindME"

[1] "index"      "datasets"   "serialized" "metadata"

[1] "stats"     "node_list" "diff_cut"

In [5]:
## the above dataset stores splicing events that has a PSI above tissue average
## the below dataset stores splicing events that has a PSI below tissue average

index@datasets

[1] "above" "below"

### 1.4 build custom index

In [24]:
## first read in all the matrix generated from running build_matrix_original.R

matrix.above = readRDS("../../processed_data/Tasic_2016/matrix_above.rds")
matrix.below = readRDS("../../processed_data/Tasic_2016/matrix_below.rds")
stats = readRDS("../../processed_data/Tasic_2016/stats.rds")
node.list = readRDS("../../processed_data/Tasic_2016/gene_node_all.rds")
diff.cut = readRDS("../../processed_data/Tasic_2016/diff_cut.rds")
meta = readRDS("../../../original_data/Tasic_2016/pseudobulk_meta.rds")

In [25]:
head(meta)

,pseudo_bulk_ID,samples,primary_type,cell_type,pseudo_bulk_ID_new
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Vip_Mybpc1_1,"SRR2139051,SRR2139022,SRR2138657,SRR2139077,SRR2139043",Vip_Mybpc1,Vip_Mybpc1,Vip_Mybpc1_1
2,Vip_Mybpc1_2,"SRR2139035,SRR2138604,SRR2139046,SRR2139057,SRR2140293",Vip_Mybpc1,Vip_Mybpc1,Vip_Mybpc1_2
3,Vip_Mybpc1_3,"SRR2139053,SRR2140329,SRR2140337,SRR2139042,SRR2138640",Vip_Mybpc1,Vip_Mybpc1,Vip_Mybpc1_3
4,Vip_Mybpc1_4,"SRR2138662,SRR2140291,SRR2140285,SRR2140281,SRR2138955",Vip_Mybpc1,Vip_Mybpc1,Vip_Mybpc1_4
5,Vip_Mybpc1_5,"SRR2138646,SRR2139545,SRR2139068,SRR2138643,SRR2138627",Vip_Mybpc1,Vip_Mybpc1,Vip_Mybpc1_5
6,Vip_Mybpc1_6,"SRR2138936,SRR2140292,SRR2139028,SRR2140279",Vip_Mybpc1,Vip_Mybpc1,Vip_Mybpc1_6


In [26]:
head(matrix.above)

,Vip_Gpc3_8,Pvalb_Wt1_10,L4_Arf5_8,Igtp_2,Sst_Cbln4_7,Vip_Sncg_2,L4_Arf5_6,Micro_Ctss_3,Sst_Cdk6_4,Oligo_Opalin_4,⋯,Vip_Chat_2,Pvalb_Tacr3_12,L5a_Pde1c_3,Pvalb_Gpx3_8,L6a_Sla_7,L2-3_Ptgs2_14,L5b_Tph2_1,Pvalb_Obox3_4,L6a_Sla_10,Pvalb_Wt1_5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000048960_51,0.00000,0,0.00000,0,0,0.00000,0,0.00000,0.00000,0.00000,⋯,0.00000,0.00000,0,0,0,0.00000,0,0.00000,0,0.00000
ENSMUSG00000048960_52,0.00000,0,0.00000,0,0,0.00000,0,0.00000,0.00000,0.00000,⋯,0.00000,0.00000,0,0,0,0.00000,0,0.00000,0,0.00000
ENSMUSG00000048960_53,0.00000,0,0.00000,0,0,0.00000,0,0.00000,0.00000,0.00000,⋯,0.00000,0.00000,0,0,0,0.00000,0,0.00000,0,0.00000
ENSMUSG00000048960_54,0.00000,0,0.00000,0,0,0.00000,0,0.00000,0.00000,0.00000,⋯,0.00000,0.00000,0,0,0,0.00000,0,0.00000,0,0.00000
ENSMUSG00000048960_55,0.00000,0,0.00000,0,0,0.00000,0,0.00000,0.00000,0.00000,⋯,0.00000,0.00000,0,0,0,0.00000,0,0.00000,0,0.00000
ENSMUSG00000026575_22,35.01955,0,35.01955,0,0,35.01955,0,35.01955,35.01955,35.01955,⋯,35.01955,35.01955,0,0,0,35.01955,0,35.01955,0,35.01955


In [27]:
levels(factor(meta$cell_type))

[1] "Astro_Gja1"    "Endo_Myl9"     "Endo_Tbc1d4"   "Igtp"         
 [5] "L2_Ngb"        "L2-3_Ptgs2"    "L4_Arf5"       "L4_Ctxn3"     
 [9] "L4_Scnn1a"     "L5_Chrna6"     "L5_Ucma"       "L5a_Batf3"    
[13] "L5a_Hsd11b1"   "L5a_Pde1c"     "L5a_Tcerg1l"   "L5b_Cdh13"    
[17] "L5b_Tph2"      "L6a_Car12"     "L6a_Mgp"       "L6a_Sla"      
[21] "L6a_Syt17"     "L6b_Rgs12"     "L6b_Serpinb11" "Micro_Ctss"   
[25] "Ndnf_Car4"     "Ndnf_Cxcl14"   "Oligo_96_Rik"  "Oligo_Opalin" 
[29] "OPC_Pdgfra"    "Pvalb_Cpne5"   "Pvalb_Gpx3"    "Pvalb_Obox3"  
[33] "Pvalb_Rspo2"   "Pvalb_Tacr3"   "Pvalb_Tpbg"    "Pvalb_Wt1"    
[37] "Smad3"         "Sncg"          "Sst_Cbln4"     "Sst_Cdk6"     
[41] "Sst_Chodl"     "Sst_Myh8"      "Sst_Tacstd2"   "Sst_Th"       
[45] "Vip_Chat"      "Vip_Gpc3"      "Vip_Mybpc1"    "Vip_Parm1"    
[49] "Vip_Sncg"

In [28]:
meta = meta[match(colnames(matrix.above), meta$pseudo_bulk_ID_new), ]

In [31]:
## make sure that the order of cell in metadata is the same with order of cell in PSI matrix
identical(meta$pseudo_bulk_ID_new, colnames(matrix.above))
identical(meta$pseudo_bulk_ID_new, colnames(matrix.below))

[1] TRUE

[1] TRUE

In [32]:
## build the index for spliced-in events, denoted by 'above' i.e. PSI is above dataset average
idx_above <- buildAltSpliceIndex(psival = matrix.above, metadata = meta, dataset.name = 'above', column.label = 'cell_type', qb = 2)

Generating index for above

	Indexing Astro_Gja1 as above.Astro_Gja1  with  9  cells.

	Indexing Endo_Myl9 as above.Endo_Myl9  with  3  cells.

	Indexing Endo_Tbc1d4 as above.Endo_Tbc1d4  with  3  cells.

	Indexing Igtp as above.Igtp  with  3  cells.

	Indexing L2_Ngb as above.L2_Ngb  with  4  cells.

	Indexing L2-3_Ptgs2 as above.L2-3_Ptgs2  with  19  cells.

	Indexing L4_Arf5 as above.L4_Arf5  with  9  cells.

	Indexing L4_Ctxn3 as above.L4_Ctxn3  with  16  cells.

	Indexing L4_Scnn1a as above.L4_Scnn1a  with  20  cells.

	Indexing L5_Chrna6 as above.L5_Chrna6  with  3  cells.

	Indexing L5_Ucma as above.L5_Ucma  with  3  cells.

	Indexing L5a_Batf3 as above.L5a_Batf3  with  13  cells.

	Indexing L5a_Hsd11b1 as above.L5a_Hsd11b1  with  11  cells.

	Indexing L5a_Pde1c as above.L5a_Pde1c  with  4  cells.

	Indexing L5a_Tcerg1l as above.L5a_Tcerg1l  with  7  cells.

	Indexing L5b_Cdh13 as above.L5b_Cdh13  with  6  cells.

	Indexing L5b_Tph2 as above.L5b_Tph2  with  5  cells.

	Indexing 

In [35]:
## build the index for spliced-out events, denoted by 'below' i.e. PSI is below dataset average
idx_below <- buildAltSpliceIndex(psival = matrix.above, metadata = meta, dataset.name = 'below', column.label = 'cell_type', qb = 2)

Generating index for below

	Indexing Astro_Gja1 as below.Astro_Gja1  with  9  cells.

	Indexing Endo_Myl9 as below.Endo_Myl9  with  3  cells.

	Indexing Endo_Tbc1d4 as below.Endo_Tbc1d4  with  3  cells.

	Indexing Igtp as below.Igtp  with  3  cells.

	Indexing L2_Ngb as below.L2_Ngb  with  4  cells.

	Indexing L2-3_Ptgs2 as below.L2-3_Ptgs2  with  19  cells.

	Indexing L4_Arf5 as below.L4_Arf5  with  9  cells.

	Indexing L4_Ctxn3 as below.L4_Ctxn3  with  16  cells.

	Indexing L4_Scnn1a as below.L4_Scnn1a  with  20  cells.

	Indexing L5_Chrna6 as below.L5_Chrna6  with  3  cells.

	Indexing L5_Ucma as below.L5_Ucma  with  3  cells.

	Indexing L5a_Batf3 as below.L5a_Batf3  with  13  cells.

	Indexing L5a_Hsd11b1 as below.L5a_Hsd11b1  with  11  cells.

	Indexing L5a_Pde1c as below.L5a_Pde1c  with  4  cells.

	Indexing L5a_Tcerg1l as below.L5a_Tcerg1l  with  7  cells.

	Indexing L5b_Cdh13 as below.L5b_Cdh13  with  6  cells.

	Indexing L5b_Tph2 as below.L5b_Tph2  with  5  cells.

	Indexing 

In [36]:
## merge the above and below index to a metaindex object
both <- mergeDataset(idx_above, idx_below)

Merging below



In [37]:
## finally add the metadata to complete the index

index_custom <- addIndexMeta(object = both, stats = stats, node_list = node.list, diff_cut = diff.cut)

In [39]:
class(index_custom)

[1] "SCFind"
attr(,"package")
[1] "scfindME"

## 2. Exploratory analysis

In [40]:
# first, check the encoded cell types
cell_types <- cellTypeNames(index)
cell_types

[1] "above.Astro_Gja1"    "above.Endo_Myl9"     "above.Endo_Tbc1d4"  
 [4] "above.Igtp"          "above.L2_Ngb"        "above.L2-3_Ptgs2"   
 [7] "above.L4_Arf5"       "above.L4_Ctxn3"      "above.L4_Scnn1a"    
[10] "above.L5_Chrna6"     "above.L5_Ucma"       "above.L5a_Batf3"    
[13] "above.L5a_Hsd11b1"   "above.L5a_Pde1c"     "above.L5a_Tcerg1l"  
[16] "above.L5b_Cdh13"     "above.L5b_Tph2"      "above.L6a_Car12"    
[19] "above.L6a_Mgp"       "above.L6a_Sla"       "above.L6a_Syt17"    
[22] "above.L6b_Rgs12"     "above.L6b_Serpinb11" "above.Micro_Ctss"   
[25] "above.Ndnf_Car4"     "above.Ndnf_Cxcl14"   "above.Oligo_96_Rik" 
[28] "above.Oligo_Opalin"  "above.OPC_Pdgfra"    "above.Pvalb_Cpne5"  
[31] "above.Pvalb_Gpx3"    "above.Pvalb_Obox3"   "above.Pvalb_Rspo2"  
[34] "above.Pvalb_Tacr3"   "above.Pvalb_Tpbg"    "above.Pvalb_Wt1"    
[37] "above.Smad3"         "above.Sncg"          "above.Sst_Cbln4"    
[40] "above.Sst_Cdk6"      "above.Sst_Chodl"     "above.Sst_Myh8"     
[43] "above.Sst_Tacstd2"   "above.Sst_Th"        "above.Vip_Chat"     
[46] "above.Vip_Gpc3"      "above.Vip_Mybpc1"    "above.Vip_Parm1"    
[49] "above.Vip_Sncg"      "below.Astro_Gja1"    "below.Endo_Myl9"    
[52] "below.Endo_Tbc1d4"   "below.Igtp"          "below.L2_Ngb"       
[55] "below.L2-3_Ptgs2"    "below.L4_Arf5"       "below.L4_Ctxn3"     
[58] "below.L4_Scnn1a"     "below.L5_Chrna6"     "below.L5_Ucma"      
[61] "below.L5a_Batf3"     "below.L5a_Hsd11b1"   "below.L5a_Pde1c"    
[64] "below.L5a_Tcerg1l"   "below.L5b_Cdh13"     "below.L5b_Tph2"     
[67] "below.L6a_Car12"     "below.L6a_Mgp"       "below.L6a_Sla"      
[70] "below.L6a_Syt17"     "below.L6b_Rgs12"     "below.L6b_Serpinb11"
[73] "below.Micro_Ctss"    "below.Ndnf_Car4"     "below.Ndnf_Cxcl14"  
[76] "below.Oligo_96_Rik"  "below.Oligo_Opalin"  "below.OPC_Pdgfra"   
[79] "below.Pvalb_Cpne5"   "below.Pvalb_Gpx3"    "below.Pvalb_Obox3"  
[82] "below.Pvalb_Rspo2"   "below.Pvalb_Tacr3"   "below.Pvalb_Tpbg"   
[85] "below.Pvalb_Wt1"     "below.Smad3"         "below.Sncg"         
[88] "below.Sst_Cbln4"     "below.Sst_Cdk6"      "below.Sst_Chodl"    
[91] "below.Sst_Myh8"      "below.Sst_Tacstd2"   "below.Sst_Th"       
[94] "below.Vip_Chat"      "below.Vip_Gpc3"      "below.Vip_Mybpc1"   
[97] "below.Vip_Parm1"     "below.Vip_Sncg"

### 2.1 Search marker nodes of an interested cell type

In [41]:
interested_cell_type <- "above.Astro_Gja1"

# rank the splicing nodes by their frequency of inclusion that is above dataset mean in the interested cell type

markers <- cellTypeMarkers(index, interested_cell_type, background.cell.types = cellTypeNames(index, datasets = 'above'), top.k = 10)
markers

,cellType,genes,tp,fp,fn,precision,recall,f1
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
9899,above.Astro_Gja1,ENSMUSG00000037196_3,8,0,1,1.0000000,0.8888889,0.9411765
13558,above.Astro_Gja1,ENSMUSG00000061080_25,9,3,0,0.7500000,1.0000000,0.8571428
13559,above.Astro_Gja1,ENSMUSG00000061080_26,9,3,0,0.7500000,1.0000000,0.8571428
8726,above.Astro_Gja1,ENSMUSG00000032826_55,7,1,2,0.8749999,0.7777778,0.8235294
7425,above.Astro_Gja1,ENSMUSG00000030235_2,9,4,0,0.6923077,1.0000000,0.8181818
1685,above.Astro_Gja1,ENSMUSG00000017390_4,8,4,1,0.6666667,0.8888889,0.7619048
9322,above.Astro_Gja1,ENSMUSG00000035104_3,6,1,3,0.8571429,0.6666667,0.7500001
13491,above.Astro_Gja1,ENSMUSG00000060424_10,7,3,2,0.7000000,0.7777778,0.7368421
13501,above.Astro_Gja1,ENSMUSG00000060477_2,7,3,2,0.7000000,0.7777778,0.7368421


In [42]:
# plot the nodes using their precision. recall and f1 scores to examine

ggplot(markers, aes(x = precision, y = recall, colour = f1, label = genes)) +
theme_bw() + 
geom_point(aes(size = f1)) +
geom_text(vjust = 0, 
          nudge_y = 0.01, 
          colour = "black", 
          check_overlap = T) +
xlim(0.4, 1.2)

fig(6, 6)

In [43]:
## get the annotations of query nodes
## use the markers of interested cell type as example
nodeDetails(index, as.character(markers$genes))

,Gene_id,Node,Coord,Strand,Type,ensembl_gene_id_version,Gene_name,Node_id,Node_name
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
298236,ENSMUSG00000037196,3,17:10839761-10839916,-,CE,ENSMUSG00000037196.7,Pacrg,ENSMUSG00000037196_3,Pacrg_3
431117,ENSMUSG00000061080,25,16:42144025-42144085,+,CE,ENSMUSG00000061080.12,Lsamp,ENSMUSG00000061080_25,Lsamp_25
431113,ENSMUSG00000061080,26,16:42144552-42144700,+,CE,ENSMUSG00000061080.12,Lsamp,ENSMUSG00000061080_26,Lsamp_26
254878,ENSMUSG00000032826,55,3:127024774-127024843,-,AD,ENSMUSG00000032826.20,Ank2,ENSMUSG00000032826_55,Ank2_55
215563,ENSMUSG00000030235,2,6:141524535-141524549,+,CE,ENSMUSG00000030235.18,Slco1c1,ENSMUSG00000030235_2,Slco1c1_2
48707,ENSMUSG00000017390,4,11:78324200-78324325,+,AA,ENSMUSG00000017390.16,Aldoc,ENSMUSG00000017390_4,Aldoc_4
278684,ENSMUSG00000035104,3,6:82041688-82041834,+,AA,ENSMUSG00000035104.15,Eva1a,ENSMUSG00000035104_3,Eva1a_3
428838,ENSMUSG00000060424,10,1:42666841-42666851,-,AD,ENSMUSG00000060424.15,Pantr1,ENSMUSG00000060424_10,Pantr1_10
429103,ENSMUSG00000060477,2,6:113638528-113638641,+,CE,ENSMUSG00000060477.15,Irak2,ENSMUSG00000060477_2,Irak2_2


### 2.2 Search cell types enriched in an interested splicing event

In [44]:
# find all nodes in a particular gene

Gria2_nodes <- geneNodes(index, "Gria2", "Gene_name")
Gria2_nodes <- Gria2_nodes[order(as.numeric(as.character(Gria2_nodes$Node))), ]
Gria2_nodes

,Gene_id,Node,Coord,Strand,Type,ensembl_gene_id_version,Gene_name,Node_id,Node_name
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
265697,ENSMUSG00000033981,3,3:80802580-80802650,-,AA,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_3,Gria2_3
265702,ENSMUSG00000033981,4,3:80802297-80802579,-,CE,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_4,Gria2_4
265695,ENSMUSG00000033981,5,3:80801648-80801785,-,NA,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_5,Gria2_5
265708,ENSMUSG00000033981,6,3:80801645-80801647,-,NA,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_6,Gria2_6
265710,ENSMUSG00000033981,7,3:80800036-80801644,-,AD,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_7,Gria2_7
265701,ENSMUSG00000033981,21,3:80706888-80707063,-,AD,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_21,Gria2_21
265704,ENSMUSG00000033981,22,3:80706353-80706887,-,AD,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_22,Gria2_22
265711,ENSMUSG00000033981,25,3:80692285-80692532,-,CE,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_25,Gria2_25
265707,ENSMUSG00000033981,26,3:80691744-80692284,-,RI,ENSMUSG00000033981.15,Gria2,ENSMUSG00000033981_26,Gria2_26


In [45]:
# query significant cell types by a specific splicing event
## accept logical operators: prepend "-" means node exclusion

Gria2_mxe <- c("ENSMUSG00000033981_28", "ENSMUSG00000033981_29")
hyperQueryCellTypes(index, c("ENSMUSG00000033981_28", "-ENSMUSG00000033981_29"), datasets = 'above') %>% arrange(pval)

Found 124 cell expressing ENSMUSG00000033981_28

Excluded 4 cells expressing ENSMUSG00000033981_29



cell_type,cell_hits,total_cells,pval
<chr>,<int>,<dbl>,<dbl>
above.Igtp,3,3,0.0000000
above.Pvalb_Obox3,4,4,0.0000000
above.Pvalb_Wt1,11,11,0.0000000
above.Smad3,3,3,0.0000000
above.Vip_Sncg,3,3,0.0000000
above.L4_Arf5,8,9,0.2292646
above.Sst_Myh8,7,8,0.4080985
above.Ndnf_Cxcl14,6,7,0.7231381
above.Astro_Gja1,1,9,1.0000000


### 3. Retrieve actual PSI for discovered splicing patterns

In [46]:
## check the raw PSI value of query nodes in particular cell types, return a dataframe with cell type, node raw PSI and node details

raw_psi = getRawPsi(object = index, node.list = Gria2_mxe, cell.types = gsub("above\\.", "", cellTypeNames(index, datasets = 'above')))
head(raw_psi)

Astro_Gja1

Added Astro_Gja1

Endo_Myl9

Added Endo_Myl9

Endo_Tbc1d4

Added Endo_Tbc1d4

Igtp

Added Igtp

L2_Ngb

Added L2_Ngb

L2-3_Ptgs2

Added L2-3_Ptgs2

L4_Arf5

Added L4_Arf5

L4_Ctxn3

Added L4_Ctxn3

L4_Scnn1a

Added L4_Scnn1a

L5_Chrna6

Added L5_Chrna6

L5_Ucma

Added L5_Ucma

L5a_Batf3

Added L5a_Batf3

L5a_Hsd11b1

Added L5a_Hsd11b1

L5a_Pde1c

Added L5a_Pde1c

L5a_Tcerg1l

Added L5a_Tcerg1l

L5b_Cdh13

Added L5b_Cdh13

L5b_Tph2

Added L5b_Tph2

L6a_Car12

Added L6a_Car12

L6a_Mgp

Added L6a_Mgp

L6a_Sla

Added L6a_Sla

L6a_Syt17

Added L6a_Syt17

L6b_Rgs12

Added L6b_Rgs12

L6b_Serpinb11

Added L6b_Serpinb11

Micro_Ctss

Added Micro_Ctss

Ndnf_Car4

Added Ndnf_Car4

Ndnf_Cxcl14

Added Ndnf_Cxcl14

Oligo_96_Rik

Added Oligo_96_Rik

Oligo_Opalin

Added Oligo_Opalin

OPC_Pdgfra

Added OPC_Pdgfra

Pvalb_Cpne5

Added Pvalb_Cpne5

Pvalb_Gpx3

Added Pvalb_Gpx3

Pvalb_Obox3

Added Pvalb_Obox3

Pvalb_Rspo2

Added Pvalb_Rspo2

Pvalb_Tacr3

Added Pvalb_Tacr3

Pvalb_Tpbg

Added Pval

,Node_id,Cell_id,raw_psi,Gene_id,Node,Coord,Strand,Type,ensembl_gene_id_version,Gene_name,Node_name
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ENSMUSG00000033981_28,Astro_Gja1_1,1,ENSMUSG00000033981,28,3:80691521-80691702,-,AD,ENSMUSG00000033981.15,Gria2,Gria2_28
2,ENSMUSG00000033981_28,Astro_Gja1_2,NA,ENSMUSG00000033981,28,3:80691521-80691702,-,AD,ENSMUSG00000033981.15,Gria2,Gria2_28
3,ENSMUSG00000033981_28,Astro_Gja1_3,1,ENSMUSG00000033981,28,3:80691521-80691702,-,AD,ENSMUSG00000033981.15,Gria2,Gria2_28
4,ENSMUSG00000033981_28,Astro_Gja1_4,1,ENSMUSG00000033981,28,3:80691521-80691702,-,AD,ENSMUSG00000033981.15,Gria2,Gria2_28
5,ENSMUSG00000033981_28,Astro_Gja1_5,NA,ENSMUSG00000033981,28,3:80691521-80691702,-,AD,ENSMUSG00000033981.15,Gria2,Gria2_28
6,ENSMUSG00000033981_28,Astro_Gja1_6,NA,ENSMUSG00000033981,28,3:80691521-80691702,-,AD,ENSMUSG00000033981.15,Gria2,Gria2_28


### 4. Heatmap of PSI value of a splicing pattern

In [48]:
## generate a heatmap for raw PSI values

plotRawPsiHeatmap(raw_psi = raw_psi, node.list = Gria2_mxe, cell.types = gsub("above\\.", "", cellTypeNames(index, datasets = 'above')))

fig(6, 10)

In [49]:
## find nodes with a similar splcing pattern with query nodes

findSimilarNodes(object = index, gene.list = Gria2_mxe[1], datasets = 'above', top.k = 10)

Searching for genes with similar pattern...



  |======================================================================| 100%


gene,Jaccard,overlap,n
<chr>,<dbl>,<dbl>,<dbl>
ENSMUSG00000026554_38,0.3565217,82,188
ENSMUSG00000009681_31,0.3571429,80,180
ENSMUSG00000050587_2,0.3571429,75,161
ENSMUSG00000032479_18,0.3628319,82,184
ENSMUSG00000037492_3,0.3631841,73,150
ENSMUSG00000061751_100,0.3641304,67,127
ENSMUSG00000044647_21,0.3647059,62,108
ENSMUSG00000002280_25,0.3750000,81,173
ENSMUSG00000066392_38,0.3811881,77,155


In [50]:
above_glul5 = cell_types[grep("above\\.L5", cell_types)]

In [51]:
above_glul5

[1] "above.L5_Chrna6"   "above.L5_Ucma"     "above.L5a_Batf3"  
[4] "above.L5a_Hsd11b1" "above.L5a_Pde1c"   "above.L5a_Tcerg1l"
[7] "above.L5b_Cdh13"   "above.L5b_Tph2"

In [53]:
# find house keeping nodes in a particular group of cell types
## use L5 glutamatergic neurons as example

above_glul5_nodes = findHouseKeepingNodes(object = index, cell.types = above_glul5)
above_glul5_nodes

Searching for house keeping node...



  |======================================================================| 100%


[1] "ENSMUSG00000002825_7"  "ENSMUSG00000023960_2"  "ENSMUSG00000001211_34"
[4] "ENSMUSG00000007739_4"  "ENSMUSG00000019951_38" "ENSMUSG00000020111_21"

In [54]:
nodeDetails(object = index, node.list = above_glul5_nodes)

,Gene_id,Node,Coord,Strand,Type,ensembl_gene_id_version,Gene_name,Node_id,Node_name
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
13004,ENSMUSG00000002825,7,9:21412233-21412301,+,CE,ENSMUSG00000002825.9,Qtrt1,ENSMUSG00000002825_7,Qtrt1_7
115379,ENSMUSG00000023960,2,17:44078899-44078982,+,AD,ENSMUSG00000023960.14,Enpp5,ENSMUSG00000023960_2,Enpp5_2
5450,ENSMUSG00000001211,34,10:78282477-78283094,-,RI,ENSMUSG00000001211.16,Agpat3,ENSMUSG00000001211_34,Agpat3_34
30845,ENSMUSG00000007739,4,11:22993267-22993319,+,CE,ENSMUSG00000007739.11,Cct4,ENSMUSG00000007739_4,Cct4_4
60233,ENSMUSG00000019951,38,10:89818183-89818288,+,AD,ENSMUSG00000019951.11,Uhrf1bp1l,ENSMUSG00000019951_38,Uhrf1bp1l_38
62735,ENSMUSG00000020111,21,10:59762413-59762656,+,CE,ENSMUSG00000020111.16,Micu1,ENSMUSG00000020111_21,Micu1_21


In [55]:
## find signature splicing events for particular cell types

findNodeSignatures(object = index, cell.types = above_glul5)

Searching for node signatures...



  |======================================================================| 100%


$above.L5a_Batf3
[1] "ENSMUSG00000016763_29" "ENSMUSG00000017493_15" "ENSMUSG00000024646_1" 
[4] "ENSMUSG00000025289_3"  "ENSMUSG00000025393_28" "ENSMUSG00000029672_17"
[7] "ENSMUSG00000037624_21"

$above.L5a_Hsd11b1
[1] "ENSMUSG00000024505_11" "ENSMUSG00000026514_12" "ENSMUSG00000032336_9" 
[4] "ENSMUSG00000037624_21" "ENSMUSG00000045671_11" "ENSMUSG00000066392_35"

In [56]:
## find all cells per cell type that express the full set of nodes in gene.list

findCellTypes(object = index, gene.list = Gria2_mxe[1], datasets = 'above')

$above.Pvalb_Gpx3
 [1]  1  2  3  5  6  7  8  9 10 12

$above.OPC_Pdgfra
[1] 4

$above.Ndnf_Cxcl14
[1] 1 2 3 4 5 6

$above.L4_Arf5
[1] 1 2 3 4 5 6 7 8

$above.L5a_Tcerg1l
[1] 1

$above.L5a_Pde1c
[1] 2

$above.Ndnf_Car4
[1] 1 2 6

$above.L4_Scnn1a
 [1]  1  2  3  4  5  7  8 11 15 16 18

$above.Pvalb_Tacr3
[1]  1  2  4  5  7  9 10 12 13

$above.L5a_Batf3
[1]  3  5  7 10 12

$above.L5_Chrna6
[1] 2

$above.Vip_Parm1
[1] 1 3 5

$above.Astro_Gja1
[1] 1 3 4 8

$above.L4_Ctxn3
[1]  3  4  7  8 11 13 15 16

$above.L5a_Hsd11b1
[1]  3  4  6 10

$above.Igtp
[1] 1 2 3

$above.Pvalb_Obox3
[1] 1 2 3 4

$above.Smad3
[1] 1 2 3

$above.Sst_Th
[1] 1 2

$above.Pvalb_Wt1
 [1]  1  2  3  4  5  6  7  8  9 10 11

$above.Sst_Myh8
[1] 1 2 3 4 5 6 8

$above.Sst_Cdk6
[1] 2 3 4

$above.Vip_Chat
[1]  4  5  6  9 10

$above.Vip_Gpc3
[1] 4

$above.Vip_Sncg
[1] 1 2 3

$above.Vip_Mybpc1
[1] 1 5

$above.Pvalb_Rspo2
[1] 1 2 4 5 6

In [62]:
## TODO
## long query optimization routine

markerNodes(object = index, gene.list = c(Gria2_nodes$Node_id), datasets = c('above'))

ERROR: Error in object@index$findMarkerGenes(as.character(caseCorrect(object, : Not compatible with STRSXP: [type=NULL].


In [58]:
markerNodes(object = index, gene.list = c(Gria2_nodes$Node_id))

calculating tfidf for the reduced expression matrix... 


Genes,Query,tfidf,Cells
<int>,<chr>,<dbl>,<int>
2,"ENSMUSG00000033981_21,ENSMUSG00000033981_25",0.03138982,48
3,"ENSMUSG00000033981_21,ENSMUSG00000033981_25,ENSMUSG00000033981_26",0.04636023,47
4,"ENSMUSG00000033981_21,ENSMUSG00000033981_25,ENSMUSG00000033981_26,ENSMUSG00000033981_28",0.06032532,23
4,"ENSMUSG00000033981_21,ENSMUSG00000033981_25,ENSMUSG00000033981_26,ENSMUSG00000033981_29",0.06263539,9
3,"ENSMUSG00000033981_21,ENSMUSG00000033981_25,ENSMUSG00000033981_28",0.04555008,24
3,"ENSMUSG00000033981_21,ENSMUSG00000033981_25,ENSMUSG00000033981_29",0.04766498,9
2,"ENSMUSG00000033981_21,ENSMUSG00000033981_26",0.03143161,51
3,"ENSMUSG00000033981_21,ENSMUSG00000033981_26,ENSMUSG00000033981_28",0.04556339,24
3,"ENSMUSG00000033981_21,ENSMUSG00000033981_26,ENSMUSG00000033981_29",0.04772144,10


In [67]:

markerNodes(index, geneNodes(index, 'Gria2',query.type = "Gene_name")$Node_id) %>%
    arrange(desc(tfidf)) %>%
    slice_head(n = 30) %>%
    filter(Cells == Mode(Cells)) %>%
    slice_max(Genes, n = 5)


calculating tfidf for the reduced expression matrix... 


Genes,Query,tfidf,Cells
<int>,<chr>,<dbl>,<int>
4,"ENSMUSG00000033981_22,ENSMUSG00000033981_25,ENSMUSG00000033981_26,ENSMUSG00000033981_7",0.06030977,6
4,"ENSMUSG00000033981_21,ENSMUSG00000033981_29,ENSMUSG00000033981_31,ENSMUSG00000033981_4",0.05362449,6
4,"ENSMUSG00000033981_25,ENSMUSG00000033981_26,ENSMUSG00000033981_29,ENSMUSG00000033981_4",0.04935919,6
3,"ENSMUSG00000033981_25,ENSMUSG00000033981_26,ENSMUSG00000033981_33",0.04986312,6
3,"ENSMUSG00000033981_25,ENSMUSG00000033981_26,ENSMUSG00000033981_32",0.04906834,6
3,"ENSMUSG00000033981_22,ENSMUSG00000033981_25,ENSMUSG00000033981_7",0.04553755,6


In [70]:
object = index
gene.list = c("Gria1", "Gria2")
query.type = 'Gene_name'
node.types = c("CE", "AD", "AA", "RI", NA, "NA")


  nodes <- gene.nodes(object, gene.list, query.type)
  nodes.new <- nodes[which(as.character(nodes$Type) %in% node.types), ]

In [71]:
nodes.new

,Gene_id,Node,Coord,Strand,Type,ensembl_gene_id_version,Gene_name,Node_id,Node_name
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
69088,ENSMUSG00000020524,15,11:57221018-57221185,+,CE,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_15,Gria1_15
69089,ENSMUSG00000020524,23,11:57310608-57310722,+,CE,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_23,Gria1_23
69095,ENSMUSG00000020524,3,11:57011976-57012073,+,AA,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_3,Gria1_3
69097,ENSMUSG00000020524,25,11:57327443-57330240,+,NA,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_25,Gria1_25
69098,ENSMUSG00000020524,5,11:57013046-57013170,+,CE,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_5,Gria1_5
69100,ENSMUSG00000020524,2,11:57011810-57011975,+,AA,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_2,Gria1_2
69101,ENSMUSG00000020524,22,11:57309769-57309883,+,CE,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_22,Gria1_22
69102,ENSMUSG00000020524,21,11:57289248-57289495,+,CE,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_21,Gria1_21
69103,ENSMUSG00000020524,24,11:57317666-57317800,+,CE,ENSMUSG00000020524.17,Gria1,ENSMUSG00000020524_24,Gria1_24


In [77]:
as.character(nodes.new$Node_id)

[1] "ENSMUSG00000020524_15" "ENSMUSG00000020524_23" "ENSMUSG00000020524_3" 
 [4] "ENSMUSG00000020524_25" "ENSMUSG00000020524_5"  "ENSMUSG00000020524_2" 
 [7] "ENSMUSG00000020524_22" "ENSMUSG00000020524_21" "ENSMUSG00000020524_24"
[10] "ENSMUSG00000020524_13" "ENSMUSG00000033981_31" "ENSMUSG00000033981_28"
[13] "ENSMUSG00000033981_5"  "ENSMUSG00000033981_3"  "ENSMUSG00000033981_30"
[16] "ENSMUSG00000033981_21" "ENSMUSG00000033981_4"  "ENSMUSG00000033981_32"
[19] "ENSMUSG00000033981_22" "ENSMUSG00000033981_26" "ENSMUSG00000033981_6" 
[22] "ENSMUSG00000033981_33" "ENSMUSG00000033981_7"  "ENSMUSG00000033981_25"
[25] "ENSMUSG00000033981_29"

In [80]:
markerNodes(object, as.character(nodes.new$Node_id), datasets = 'above')

ERROR: Error in object@index$findMarkerGenes(as.character(caseCorrect(object, : Not compatible with STRSXP: [type=NULL].


In [81]:
  markers <- markerNodes(object, as.character(nodes.new$Node_id))
  if (nrow(markers) == 0) stop("No gene pattern is found")
  sets <- data.frame()

calculating tfidf for the reduced expression matrix... 


In [85]:
query

[1] "ENSMUSG00000020524_22" "ENSMUSG00000033981_22" "ENSMUSG00000033981_25"
[4] "ENSMUSG00000033981_26" "ENSMUSG00000033981_29"

In [83]:
  query <- strsplit(as.character(markers[which.max(markers$tfidf), "Query"]), ",")[[1]]
  result <- hyperQueryCellTypes(object, query)

  print("running hyperQueryCellTypeAS using")
  print(query)
  print(result)


  for (j in seq(1, nrow(result))) {
    if (result$pval[[j]] <= 0.05) {
      print("find a cell type specific node set\n")
      print(query)
      sets <- rbind(sets, result[j, ])
    }
  }

  if (nrow(sets) == 0) {
    message("This query is not cell type specific")
  }

[1] "running hyperQueryCellTypeAS using"
[1] "ENSMUSG00000020524_22" "ENSMUSG00000033981_22" "ENSMUSG00000033981_25"
[4] "ENSMUSG00000033981_26" "ENSMUSG00000033981_29"
        cell_type cell_hits total_cells pval
1 above.Sst_Cbln4         2          13    1
2 above.Sst_Chodl         2           8    1


This query is not cell type specific



## Session info

In [38]:
sessionInfo()

R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 8

Matrix products: default
BLAS/LAPACK: /nfs/research/icortes/ysong/anaconda3/envs/maftools/lib/libopenblasp-r0.3.12.so

locale:
 [1] LC_CTYPE=en_GB.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_GB.UTF-8        LC_COLLATE=en_GB.UTF-8    
 [5] LC_MONETARY=en_GB.UTF-8    LC_MESSAGES=en_GB.UTF-8   
 [7] LC_PAPER=en_GB.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_GB.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] repr_1.1.3     ggplot2_3.3.5  tidyr_1.2.0    dplyr_1.0.8    tibble_3.1.6  
[6] scfindME_0.1.0

loaded via a namespace (and not attached):
  [1] colorspace_2.0-3            ellipsis_0.3.2             
  [3] rprojroot_2.0.2             IRdisplay_1.0              
  [5] htmlTable_2.4.0     